# 位相キックバック

ユニタリゲート $U$ と、その固有状態 $\lvert\psi\rangle$ を考えます。  
ユニタリゲートの性質より、固有値はある位相角 $\theta$ を用いて、$e^{i\theta}$ と表されます。

$$
U\lvert\psi\rangle = e^{i \theta} \lvert\psi\rangle
$$

この $\theta$ を求めるとしましょう。グローバル位相として現れているため、直接測定することは難しいです。  

位相キックバックは、$\lvert\psi\rangle$ とは別の量子ビットに $\theta$ の情報を取り出して測定するテクニックです。  
そのためにはユニタリゲート $U$ を制御ゲート化した $CU$ を用います。

簡単な例を考えてみましょう。

まず、ユニタリゲートを次のように設定します。
$$
U(\theta) = \begin{pmatrix}
1 & 0 \\
0 & e^{i\theta} \\
\end{pmatrix}
$$

このとき、$\lvert 1 \rangle$ が固有状態になります。固有値は $e^{i\theta}$ です。

$$
U(\theta)\lvert 1 \rangle = e^{i\theta}\lvert 1 \rangle
$$

制御ユニタリゲートは次のようになります。
$$
CU(\theta) = CR(\theta) = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i\theta} \\
\end{pmatrix}
$$

位相キックバックでは次のような量子回路を用います。  

<img src="./img/120_img.png" width="40%">

まず、補助量子ビットに $H$ゲートを作用させて重ね合わせ状態を作ります。

$$
\lvert\psi_1\rangle = \frac{1}{\sqrt{2}}(\lvert0\rangle + \lvert1\rangle) \otimes \lvert1\rangle
$$

次に、補助量子ビットを制御量子ビットとして固有状態に $U(\theta)$ を作用させます。  
これにより、固有値が係数として取り出せます。

$$
\begin{align}
\lvert\psi_2\rangle &= \frac{1}{\sqrt{2}}(\lvert0\rangle \otimes \lvert1\rangle + \lvert1\rangle \otimes U(\theta)\lvert1\rangle) \\
&= \frac{1}{\sqrt{2}}(\lvert0\rangle \otimes \lvert1\rangle + \lvert1\rangle \otimes e^{i\theta}\lvert1\rangle) \\
&= \frac{1}{\sqrt{2}}(\lvert0\rangle \otimes \lvert1\rangle + e^{i\theta}\lvert1\rangle \otimes \lvert1\rangle) \\
&= \frac{1}{\sqrt{2}}(\lvert0\rangle + e^{i\theta} \lvert1\rangle) \otimes \lvert1\rangle)
\end{align}
$$

以上のように、$\theta$ を補助量子ビットの重ね合わせ状態の係数として取り出せました。  

ここで補助量子ビットだけに注目します。  
このまま補助量子ビットを測定しても $\lvert0\rangle$, $\lvert1\rangle$ が等確率なので、再度 $H$ゲートを作用させます。

$$
\begin{align}
H\cdot \frac{1}{\sqrt{2}}(\lvert0\rangle + e^{i\theta} \lvert1\rangle) &= \frac{1}{\sqrt{2}} 
\begin{pmatrix} 1 & 1
\\ 1 & -1 \\
\end{pmatrix}
\cdot
\frac{1}{\sqrt{2}}
\begin{pmatrix} 
1 \\ e^{i\theta} \\ 
\end{pmatrix} \\
&= \frac{1}{2} 
\begin{pmatrix} 
1 + e^{i\theta} \\ 1 - e^{i\theta} \\ 
\end{pmatrix}
\end{align}
$$

$H$ ゲートを用いて相対位相の差を振幅の差へ変換することができました。  
最後に補助量子ビットを繰り返し測定すると、$\lvert0\rangle$ を測定する確率 $\rm{Pr}(\lvert0\rangle)$ は

$$
\begin{align}
\mathrm{Pr}(\lvert0\rangle) &= \biggl| \frac{1+e^{i\theta}}{2} \biggr|^2 \\
&= \frac{1}{2}(1 + \cos\theta)
\end{align}
$$

以上より、固有値の実部を推定することができます。

以上をblueqatで実装してみましょう。

In [3]:
from blueqat import Circuit
import numpy as np

上記の量子回路を実装します。  
$\theta$ は乱数で決めておきます。

In [47]:
theta = np.random.rand() * np.pi
shots = 1024

c = Circuit(2)

c.x[1].h[0].cr(theta)[0, 1].h[0].m[0]
res = c.run(shots = shots)

実行結果から、補助量子ビットで$\lvert 0 \rangle$を測定した割合を求めます。  
それより計算した推定値 $\theta_{est}$ と、事前に乱数で決めた $\theta$ を比較します。

In [48]:
p0 = res['00'] / shots
theta_est = np.arccos(2 * p0 - 1)

In [56]:
print("θ推定値　 :", theta_est)
print("実際の値 :", theta)

θ推定値　 : 0.3718944104475685
実際の値 : 0.36989050621867126


以上より、位相キックバックで $\theta$ を推定できました。  
推定の精度は、ショット数に依存します。